In [1]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


import sys
sys.path.append('../../baseline_model/')
from train_baseline_model import ConditionDataset, load_dataset
from test_baseline_model_ood import load_test_dataset, load_dataset_label
import yaml

final_condition_data_path = '../../dataset/source_dataset/USPTO_condition_final/'
_, label_dict = load_dataset_label(
    final_condition_data_path, 'USPTO_condition', use_temperature=False)

Loading condition data from ../../dataset/source_dataset/USPTO_condition_final/
Loading database dataframe...
Loading label dict...
Condition name: catalyst1, categories: 54
########################################################
Condition name: solvent1, categories: 85
########################################################
Condition name: solvent2, categories: 41
########################################################
Condition name: reagent1, categories: 223
########################################################
Condition name: reagent2, categories: 95
########################################################


In [2]:
def get_condition_smiles(condition_name, label_dict):
    return list(label_dict[condition_name][0][0].values())

def draw_condition(smiles_list, molsPerRow=3, subImgSize=(150, 150)):
    smiles_list = [x for x in smiles_list if x]
    print(f'show {len(smiles_list)} molecules')
    mols = [Chem.MolFromSmiles(x) for x in smiles_list]
    return display(Draw.MolsToGridImage(mols, molsPerRow=molsPerRow, 
                               subImgSize=subImgSize,
                               legends=smiles_list,
                               maxMols=999999
                               ))
def canonicalize_smiles(smi):
    mol = Chem.MolFromSmiles(smi)
    return Chem.MolToSmiles(mol)

In [4]:
from urllib.request import urlopen
from urllib.parse import quote
from tqdm import tqdm
import json

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return ''


In [5]:
# Driver M D, Hunter C A. Solvent similarity index[J]. Physical Chemistry Chemical Physics, 2020, 22(21): 11967-11975.
# Dendrogram for solvents 14 parts
import os
from collections import defaultdict

convert_erro_solvent_classfication_ssi = defaultdict(list)
solvent_classfication_ssi = defaultdict(list)
dendrogram_cls_save_path = './condition_classfication_data/'
solvent_classfication_ssi_save_path = os.path.join(dendrogram_cls_save_path, 'solvent_classfication_ssi.json')
convert_erro_solvent_classfication_ssi_save_path = os.path.join(dendrogram_cls_save_path, 'convert_erro_solvent_classfication_ssi.json')
if not os.path.exists(solvent_classfication_ssi_save_path):
    files_list =[x for x in os.listdir(dendrogram_cls_save_path) if x.split('.')[-1] == 'txt']
    for name_cls in tqdm(files_list):
        with open(os.path.join(dendrogram_cls_save_path, name_cls), 'r', encoding='utf-8') as f:
            data = [x.strip() for x in f.readlines()]
        smiles_data = []
        for solvent_name in data:
            solvent_smiles = CIRconvert(solvent_name)
            if solvent_smiles:
                solvent_classfication_ssi[name_cls.split('.')[0]].append(solvent_smiles)
                print('{}: {} --> {}'.format(name_cls.split('.')[0], solvent_name, solvent_smiles))
            else:
                convert_erro_solvent_classfication_ssi[name_cls.split('.')[0]].append(solvent_name)
                print('convert fail in {}: {}!'.format(name_cls.split('.')[0], solvent_name))
    with open(solvent_classfication_ssi_save_path, 'w', encoding='utf-8') as f:
        json.dump(solvent_classfication_ssi, f)
    with open(convert_erro_solvent_classfication_ssi_save_path, 'w', encoding='utf-8') as f:
        json.dump(convert_erro_solvent_classfication_ssi, f)
else:
    with open(solvent_classfication_ssi_save_path, 'r', encoding='utf-8') as f:
        solvent_classfication_ssi = json.load(f)
    with open(convert_erro_solvent_classfication_ssi_save_path, 'r', encoding='utf-8') as f:
        convert_erro_solvent_classfication_ssi = json.load(f)


In [6]:
# dendrogram_cls_save_path = './condition_classfication_data/'
# solvent_classfication_ssi_save_path = os.path.join(dendrogram_cls_save_path, 'solvent_classfication_ssi.json')
# convert_erro_solvent_classfication_ssi_save_path = os.path.join(dendrogram_cls_save_path, 'convert_erro_solvent_classfication_ssi.json')
# with open(solvent_classfication_ssi_save_path, 'w', encoding='utf-8') as f:
#     json.dump(solvent_classfication_ssi, f)
# with open(convert_erro_solvent_classfication_ssi_save_path, 'w', encoding='utf-8') as f:
#     json.dump(convert_erro_solvent_classfication_ssi, f)

In [7]:
canonical_solvent_classfication_ssi = defaultdict(list)
keys = list(solvent_classfication_ssi.keys())
keys.sort()
for solvent_cls in keys:
    solvent_cls_smiles_list = solvent_classfication_ssi[solvent_cls]
    solvent_cls_smiles_list = [canonicalize_smiles(x) for x in solvent_cls_smiles_list]
    canonical_solvent_classfication_ssi[solvent_cls] += solvent_cls_smiles_list
    

In [8]:
# canonical_solvent_classfication_ssi
solvent2cls = {}
for solvent_cls,  solvent_cls_smiles_list in canonical_solvent_classfication_ssi.items():
    for smi in solvent_cls_smiles_list:
        solvent2cls[smi] = solvent_cls
solvent2cls

{'OCC(O)CO': 'solvent_cls_1',
 'OCCCO': 'solvent_cls_1',
 'CN(C)P(=O)(N(C)C)N(C)C': 'solvent_cls_1',
 'Cc1ccccc1': 'solvent_cls_10',
 'Cc1cccc(C)c1': 'solvent_cls_10',
 'Cc1ccc(C)cc1': 'solvent_cls_10',
 'Cc1ccccc1C': 'solvent_cls_10',
 'Cc1cc(C)cc(C)c1': 'solvent_cls_10',
 'c1ccc2c(c1)CCCC2': 'solvent_cls_10',
 'CC(C)c1ccccc1': 'solvent_cls_10',
 'CCc1ccccc1': 'solvent_cls_10',
 'C=Cc1ccccc1': 'solvent_cls_10',
 'c1ccccc1': 'solvent_cls_10',
 'c1ccc(Oc2ccccc2)cc1': 'solvent_cls_10',
 'c1ccoc1': 'solvent_cls_10',
 'C1CCSCC1': 'solvent_cls_10',
 'CCSCC': 'solvent_cls_10',
 'COc1ccccc1OC': 'solvent_cls_10',
 'C1CCSC1': 'solvent_cls_10',
 'CSC': 'solvent_cls_10',
 'CCOc1ccccc1': 'solvent_cls_10',
 'COc1ccccc1': 'solvent_cls_10',
 'CN(C)c1ccccc1': 'solvent_cls_10',
 'CC(C)SC(C)C': 'solvent_cls_10',
 'CCCCSCCCC': 'solvent_cls_10',
 'c1ccc(COCc2ccccc2)cc1': 'solvent_cls_10',
 'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F': 'solvent_cls_11',
 'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F

In [9]:
cls_cnt_dict = defaultdict(int)
uspto_solvent_to_cls = {}
unk_idx = 0
for solvent_smiles in solvent_list:
    if solvent_smiles in solvent2cls:
        uspto_solvent_to_cls[solvent_smiles] = solvent2cls[solvent_smiles]
        cls_cnt_dict[solvent2cls[solvent_smiles]] += 1
    else:
        uspto_solvent_to_cls[solvent_smiles] = f'unk{unk_idx}'
        cls_cnt_dict[f'unk{unk_idx}'] += 1
        unk_idx += 1

In [10]:
cls_cnt_dict

defaultdict(int,
            {'unk0': 1,
             'unk1': 1,
             'solvent_cls_12': 5,
             'solvent_cls_6': 8,
             'unk2': 1,
             'solvent_cls_9': 5,
             'solvent_cls_3': 9,
             'solvent_cls_8': 7,
             'unk3': 1,
             'solvent_cls_2': 15,
             'solvent_cls_4': 2,
             'solvent_cls_11': 6,
             'unk4': 1,
             'solvent_cls_1': 2,
             'unk5': 1,
             'unk6': 1,
             'solvent_cls_10': 7,
             'solvent_cls_7': 2,
             'unk7': 1,
             'solvent_cls_13': 1,
             'solvent_cls_14': 2,
             'unk8': 1,
             'unk9': 1,
             'unk10': 1,
             'unk11': 1,
             'unk12': 1,
             'unk13': 1,
             'unk14': 1,
             'unk15': 1})

In [11]:
uspto_solvent_cls_name2cls_idx_dict = {name:idx for idx, name in enumerate(list(cls_cnt_dict.keys()))}

In [12]:
uspto_solvent_cls_name2cls_idx_dict

{'unk0': 0,
 'unk1': 1,
 'solvent_cls_12': 2,
 'solvent_cls_6': 3,
 'unk2': 4,
 'solvent_cls_9': 5,
 'solvent_cls_3': 6,
 'solvent_cls_8': 7,
 'unk3': 8,
 'solvent_cls_2': 9,
 'solvent_cls_4': 10,
 'solvent_cls_11': 11,
 'unk4': 12,
 'solvent_cls_1': 13,
 'unk5': 14,
 'unk6': 15,
 'solvent_cls_10': 16,
 'solvent_cls_7': 17,
 'unk7': 18,
 'solvent_cls_13': 19,
 'solvent_cls_14': 20,
 'unk8': 21,
 'unk9': 22,
 'unk10': 23,
 'unk11': 24,
 'unk12': 25,
 'unk13': 26,
 'unk14': 27,
 'unk15': 28}

In [13]:
uspto_solvent_to_cls    # 溶剂分组
uspto_solvent_to_cls_idx = {}
for solvent in uspto_solvent_to_cls:
    uspto_solvent_to_cls_idx[solvent] = uspto_solvent_cls_name2cls_idx_dict[uspto_solvent_to_cls[solvent]]

In [14]:
uspto_solvent_to_cls_idx
with open('./condition_classfication_data/uspto_solvent_to_cls_idx.json', 'w', encoding='utf-8') as f:
    json.dump(uspto_solvent_to_cls_idx, f)

In [15]:
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
def is_similar_reagent(rgt1, rgt2):
	if rgt1 == rgt2:
		return True

	else:
		try:
			mol1 = Chem.MolFromSmiles(rgt1)
			mol2 = Chem.MolFromSmiles(rgt2)
			fp1 = FingerprintMols.FingerprintMol(mol1)
			fp2 = FingerprintMols.FingerprintMol(mol2)
		except:
			print('cannot calculate fp')
			return False
		if not any(list(fp1)) or not any(list(fp2)):
			return False
		similarity = DataStructs.FingerprintSimilarity(fp1,fp2)
		if similarity >=1.0:
			return True
		else:
			return False

In [16]:
is_similar_reagent('Cc1ccccc1','CCc1ccccc1')

False

In [17]:
import numpy as np
features_smarts = [
    '[!H0;#7,#8,#9]', # 氢键供体
    '[!$([#6,H0,-,-2,-3])]', # 氢键供体
    '[!$([#6,F,Cl,Br,I,o,s,nX3,#7v5,#15v5,#16v4,#16v6,*+1,*+2,*+3])]',     # 氢键受体
    '[#6,#7;R0]=[#8]', # 氢键受体
    
    '[CX3](=[OX1])[F,Cl,Br,I]', # 酰卤化合物
    '[$([#16X4](=[OX1])(=[OX1])([#6])[OX2H,OX1H0-]),$([#16X4+2]([OX1-])([OX1-])([#6])[OX2H,OX1H0-])]', # 磺酸
    '[$(P(=[OX1])([$([OX2H]),$([OX1-]),$([OX2]P)])([$([OX2H]),$([OX1-]),$([OX2]P)])[$([OX2H]),$([OX1-]),$([OX2]P)]),$([P+]([OX1-])([$([OX2H]),$([OX1-]),$([OX2]P)])([$([OX2H]),$([OX1-]),$([OX2]P)])[$([OX2H]),$([OX1-]),$([OX2]P)])]', #磷酸
    '[$([OH]-*=[!#6])]', # 羟基
    '[CX3](=O)[OX2H1]', # 羧酸
    '[$([CX3]=[OX1]),$([CX3+]-[OX1-])]', # 羰基
    '[CX3](=[OX1])C',
    '[OX1]=CN',
    '[CX3](=[OX1])O',
    '[CX3](=[OX1])[F,Cl,Br,I]',
    '[CX3H1](=O)[#6]',
    '[CX3](=[OX1])[OX2][CX3](=[OX1])', # 酸酐
    '[NX3][CX3](=[OX1])[#6]', # 酰胺
    '[NX3][CX3]=[NX3+]',
    '[NX3,NX4+][CX3](=[OX1])[OX2,OX1-]',
    '[CX3](=O)[O-]', # 羧酸盐离子
    '[CX3](=[OX1])(O)O', # 碳酸或碳酸脂
    '[CX3](=O)[OX1H0-,OX2H1]',  #羧酸或共轭碱
    '[NX2-]',
    '[OX2H+]=*',
    '[OX3H2+]',
    '[#6+]',
    '[$([cX2+](:*):*)]',
    '[$([NX1-]=[NX2+]=[NX1-]),$([NX1]#[NX2+]-[NX1-2])]',
    '[+1]~*~*~[-1]',
    '[+]', # 阳离子
    '[-]', # 阴离子
    '[$([cX3](:*):*),$([cX2+](:*):*)]', # 芳香SP2碳
    '[$([cX3](:*):*),$([cX2+](:*):*),$([CX3]=*),$([CX2+]=*)]', #任何SP2碳
    '[C]', # 包含碳
    '[N]', # 包含氮
    '[O]', # 包含氧
    '[P]',
    '[c]',
    '[n]',
    '[S]',
    '[B]',
    '[Na,K,Li]',
    '[Si]',
    
    
    '[O;H1]-[C,S]([O,S,P])', # '[O;H1]-[C,S](4[O,S,P])',
    '[F,Cl,Br,I]',           # 卤素
    '[$([A;D3](@*)(@*)~*)]', #
    '[$([a;D3](@*)(@*)*)]',
    '[!#1;!#6;!#7;!#8;!#9;!#16;!#17;!#35;!#53]',  
    ]
features_patts = [Chem.MolFromSmarts(x) for x in features_smarts]

def calculate_reagent_fps(mol, features_patts):

    fp = np.asarray([mol.HasSubstructMatch(patt) for patt in features_patts], dtype='int8')
    
    return fp

def calculate_reagent_similarity_with_fp(fp1, fp2):
    if not any(list(fp1)) or not any(list(fp2)):
        return False
    return (fp1 == fp2).all()

def calculate_reagent_similarity(smi1, smi2, features_patts=features_patts):
    # reagent feature

    try:
        mol1 = Chem.MolFromSmiles(smi1)
        mol2 = Chem.MolFromSmiles(smi2)
        fp1 = calculate_reagent_fps(mol1, features_patts=features_patts)
        fp2 = calculate_reagent_fps(mol2, features_patts=features_patts)
    except:
        return False
    
    return calculate_reagent_similarity_with_fp(fp1, fp2)

In [18]:
fp1 = calculate_reagent_fps(Chem.MolFromSmiles('[Br-].[K+]'), features_patts)
fp1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1], dtype=int8)

In [19]:
fp2 = calculate_reagent_fps(Chem.MolFromSmiles('[Br-].[Na+]'), features_patts)
fp2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1], dtype=int8)

In [20]:
calculate_reagent_similarity('[Br-].[K+]', '[Br-].[Na+]')

True

In [21]:
none_fp = ''.join(['0']*len(features_patts))
none_mark = 0
reagent_classification_with_fp = defaultdict(list)
for reagent in reagent_list:
    reagent_mol = Chem.MolFromSmiles(reagent)
    reagent_fp = calculate_reagent_fps(reagent_mol, features_patts)
    if ''.join([str(x) for x in reagent_fp.tolist()]) == none_fp:
        reagent_classification_with_fp[none_fp + str(none_mark)].append(reagent)
        none_mark += 1
    else:
        reagent_classification_with_fp[''.join([str(x) for x in reagent_fp.tolist()])].append(reagent)
    

In [22]:
len(reagent_classification_with_fp)

145

In [23]:
with open('./condition_classfication_data/reagent_classification_with_fp.json', 'w', encoding='utf-8') as f:
    json.dump(reagent_classification_with_fp, f)
# reagent_classification_with_fp

In [24]:
reagent_classification_dict = defaultdict(list)
fp_sorts = list(reagent_classification_with_fp)
fp_sorts.sort()
reagent_cls = 0
for fp in fp_sorts:
    reagent_cls += 1
    reagent_classification_dict['reagent_cls_{}'.format(reagent_cls)].extend(reagent_classification_with_fp[fp])
    

In [25]:
uspto_reagent_cls_name2cls_idx = {}
for r_cls, r_list in reagent_classification_dict.items():
    for r in r_list:
        uspto_reagent_cls_name2cls_idx[r] = int(r_cls.replace('reagent_cls_', '')) - 1

In [26]:
uspto_reagent_cls_name2cls_idx
with open('./condition_classfication_data/uspto_reagent_to_cls_idx.json', 'w', encoding='utf-8') as f:
    json.dump(uspto_reagent_cls_name2cls_idx, f)